In [ ]:
import json
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

### Data loading

In [ ]:
dataset_path = "../Dataset/Streamlines signals/streamlines_signals_1.json"

In [ ]:
flow_quantity = "U"
naca_numbers = ['maximum_camber', 'maximum_camber_position', 'maximum_thickness']

dataset = []
with open(dataset_path, 'r') as dataset_file:
  samples = json.load(dataset_file)
  for sample in samples:
    dataset.append({
        "features": sample["features"][flow_quantity],
        "labels": list(sample["naca_numbers"].values())
    })

### Shuffling the dataset

In [ ]:
# Shuffling the dataset
np.random.shuffle(dataset)

### Training and test set

In [ ]:
# Defining the training and test set splitting percentage
split_percentage = 0.8

# Computing the number of training samples according to the splitting percentage
num_training_samples = int(np.floor(split_percentage * len(dataset)))

# Extracting the training and test set
training_set, test_set = dataset[:num_training_samples], dataset[num_training_samples:]

In [ ]:
# Extracting the training features and labels
train_features = np.array([sample["features"] for sample in training_set])
train_labels = np.array([sample["labels"] for sample in training_set])

# Extracting the test features and labels
test_features = np.array([sample["features"] for sample in test_set])
test_labels = np.array([sample["labels"] for sample in test_set])

### Data normalization

In [ ]:
# Computing the mean and standard deviation of the training features
mean = train_features.mean(axis=0)
std = train_features.std(axis=0)

In [ ]:
# Normalizing the training and test features w.r.t. the training statistics
normalized_train_data = (train_features - mean) / std
normalized_test_data = (test_features - mean) / std

In [ ]:
# Plotting a random sample
plt.plot(normalized_train_data[random.choice([0, len(normalized_train_data)-1])])
plt.show()

In [ ]:
# Expanding the dimensions of the training and test features
normalized_train_features = np.expand_dims(normalized_train_data, axis=2)
normalized_test_features = np.expand_dims(normalized_test_data, axis=2)

### Building the model

In [ ]:
def buildModel():
  # Sequential model - CNN 1D
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(normalized_train_features.shape[1], normalized_train_features.shape[2])),
    keras.layers.Conv1D(filters=256, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.05),
    keras.layers.Conv1D(filters=128, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Dropout(0.05),
    keras.layers.Conv1D(filters=64, kernel_size=8, activation=tf.nn.relu),
    keras.layers.AveragePooling1D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(128, activation=tf.nn.relu),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(len(naca_numbers))
  ])

  # Compiling the model
  model.compile(loss='mse', optimizer='adam', metrics=['mae'])
  
  return model

In [ ]:
model = buildModel()
model.summary()

### Model training

In [ ]:
# Defining the number of training epochs
epochs = 200

# Early stopping with a patience of 10 epochs
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

# Training the model
history = model.fit(
    normalized_train_features, 
    train_labels,
    epochs=epochs,
    validation_split = 0.2,
    verbose = 1,
    callbacks=[early_stopping]
)

In [ ]:
# Function to plot the values of training and validation loss
def plotLoss(history):
  plt.plot(history.history['loss'], label='loss')
  plt.plot(history.history['val_loss'], label='val_loss')
  plt.ylim([0, np.max(history.history['loss'] + history.history['val_loss'])])
  plt.xlabel('Epoch')
  plt.ylabel('Error')
  plt.legend()
  plt.grid(True)

In [ ]:
plotLoss(history)

### Model evaluation

In [ ]:
# Extracting the values of loss, mean absolute error and mean square error
loss, mae = model.evaluate(normalized_test_features, test_labels, verbose = 0)

print("RESULTS")
print(f"Loss (Mean Square Error) --> {loss}")
print(f"Mean Absolute Error --> {mae}")

In [ ]:
# Computing the predictions of the test set
predictions = model.predict(normalized_test_features)

In [ ]:
# Function to plot the predicted values
def plotPredictions(test_labels, test_predictions, label, color):
  plt.scatter(test_labels, test_predictions, label=label, color=color)
  plt.xlabel('True values')
  plt.ylabel('Predictions')
  plt.axis('equal')
  plt.axis('square')
  plt.xlim([0, plt.xlim()[1]])
  plt.ylim([0, plt.ylim()[1]])
  plt.plot(test_labels, test_labels, color="black")
  plt.legend()

In [ ]:
colors = ["blue", "green", "orange"]

# Plotting the obtained results
for i in range(len(naca_numbers)):
  plotPredictions(test_labels[:,i], predictions[:,i], label=naca_numbers[i], color=colors[i])